# Simulated Annealing for a 3D Spin Lattice

## TODOS

* test annealing schedules

## Parameters & Initialization

In [ ]:
using HDF5
using StatsBase
using ProgressMeter

In [ ]:
srand(42);

### Type Definitions and Initialization

In [ ]:
type Schedule
    T::Vector{Float64} # the annealing schedule for the temperature
    B::Vector{Float64} # the annealing schedule for the magnetic field
    os::Int64 # current index in the schedule
    Schedule(T::Vector{Float64}, B::Vector{Float64}) = new(T, B, 1)
end

In [ ]:
type Parameters
    N::Int64 # number of lattice points in each direction
    Ntot::Int64 # total number of lattice points
    Ntherm::Int64 # number of sweeps for thermalization
    Nsweeps::Int64 # number of sweeps between configurations
    Nconfig::Int64 # number of configurations to save
    T::Float64 # the current temperature
    J::Float64 # ferromagnetic exchange
    Jp::Float64 # ferromagnetic exchange of correction term
    K::Float64 # DM interaction
    Kp::Float64 # DM interaction of correction term
    B::Float64 # magnetic field in 3 direction
    function Parameters(; N::Int64=30, Ntherm::Int64=20000, Nsweeps::Int64=50, Nconfig::Int64=2000, T::Float64=0.0, J::Float64=1.0, K::Float64=tan(2pi/10.0), B::Float64=0.0)
        Ntot = N^3; Jp = J/16.0; Kp = K/8.0
        new(N, Ntot, Ntherm, Nsweeps, Nconfig, T, J, Jp, K, Kp, B)
    end
    function Parameters(anneal::Schedule; N::Int64=30, Ntherm::Int64=20000, Nsweeps::Int64=50, Nconfig::Int64=2000)
        Parameters(N=N, Ntherm=Ntherm, Nsweeps=Nsweeps, Nconfig=Nconfig, T=anneal.T[1], B=anneal.B[1])
    end
end

In [ ]:
type Monitor
    Nrej::Int64 # number of overall rejected moves
    Nacc::Int64 # number of overall accepted moves
    nacc::Int64 # number of accepted moves per config
    accRate::Vector{Float64} # evolution of acceptance rate
    Monitor(pars::Parameters) = new(0, 0, 0, zeros(Float64, pars.Nconfig))
end

In [ ]:
type Results
    E::Vector{Float64} # total energy
    Etmp::Float64 # current total energy
    M::Array{Float64,2} # total magnetization
    Mtmp::Vector{Float64} # current total magnetization
    T::Vector{Float64} # temperature
    os::Int64 # current offset (monte carlo time)
    Results(pars::Parameters) = new(zeros(Float64, pars.Nconfig), 0.0, zeros(Float64, 3, pars.Nconfig), zeros(Float64, 3), zeros(Float64, pars.Nconfig), 0)
end

In [ ]:
init(pars::Parameters) = (Monitor(pars), Results(pars))

function init(anneal::Schedule)
    pars = Parameters(anneal)
    (pars, init(pars)...)
end

### Grid Initialization

In [ ]:
function uniformS2(dims::Tuple{Int64,Int64,Int64})
    phi = 2pi .* rand(dims)
    u = 2rand(dims) - 1.0
    squ = sqrt(1.0 - u.*u)
    x = squ .* cos(phi)
    y = squ .* sin(phi)
    res = zeros(3,dims...)
    # TODO: this should be doable with smart stacking and reshape
    for i=1:*(dims...)
        @inbounds res[:,i] = [x[i] y[i] u[i]]
    end
    res
end

@inline function uniformS2!(s::Vector{Float64})
    phi = 2pi*rand()
    u = 2rand() - 1.0
    squ = sqrt(1.0 - u*u)
    s[1:3] = [squ*cos(phi), squ*sin(phi), u]
    nothing
end

In [ ]:
function initRandomGrid(dims::Tuple{Int64,Int64,Int64})
    grid = uniformS2(dims)
    res.Etmp = totalEnergy(grid)
    res.Mtmp = totalMagnetization(grid)
    info("Initialized random grid.")
    grid
end

@inline initRandomGrid() = initRandomGrid((pars.N, pars.N, pars.N))

In [ ]:
@inline idx(i::Int64, N::Int64) = mod(i-1,N)+1

## Hamiltonian

In [ ]:
@inline crossX(a::Vector{Float64}, b::Vector{Float64}) = -a[3]*b[2] + a[2]*b[3]
@inline crossY(a::Vector{Float64}, b::Vector{Float64}) =  a[3]*b[1] - a[1]*b[3]
@inline crossZ(a::Vector{Float64}, b::Vector{Float64}) = -a[2]*b[1] + a[1]*b[2]

In [ ]:
@inline function DMterm(grid::Array{Float64,4}, i::Int64, j::Int64, k::Int64, os::Int64)
    const N = pars.N::Int64
    crossX(grid[:,i,j,k], grid[:,idx(i+os,N),j,k]) +
    crossY(grid[:,i,j,k], grid[:,i,idx(j+os,N),k]) +
    crossZ(grid[:,i,j,k], grid[:,i,j,idx(k+os,N)])
end

In [ ]:
function totalEnergy(grid::Array{Float64,4})
    N = pars.N
    E = 0.0
    for k=1:N, j=1:N, i=1:N
        E += - pars.B * grid[3,i,j,k] -
        pars.J  * dot(grid[:,i,j,k], grid[:,idx(i+1,N),j,k] + grid[:,i,idx(j+1,N),k] + grid[:,i,j,idx(k+1,N)]) +
             pars.Jp * dot(grid[:,i,j,k], grid[:,idx(i+2,N),j,k] + grid[:,i,idx(j+2,N),k] + grid[:,i,j,idx(k+2,N)]) -
             pars.K  * DMterm(grid, i, j, k, 1) +
             pars.Kp * DMterm(grid, i, j, k, 2)
    end
    E
end

In [ ]:
function totalMagnetization(grid::Array{Float64,4})
    N = pars.N
    M = zeros(Float64, 3)
    for i=1:pars.Ntot
        @inbounds M += grid[:,i]
    end
    M
end

In [ ]:
function energyChange(grid::Array{Float64,4}, snew::Vector{Float64}, i::Int64, j::Int64, k::Int64)
    const N = pars.N::Int64
    im1 = idx(i-1,N); im2 = idx(i-2,N); ip1 = idx(i+1,N); ip2 = idx(i+2,N)
    jm1 = idx(j-1,N); jm2 = idx(j-2,N); jp1 = idx(j+1,N); jp2 = idx(j+2,N)
    km1 = idx(k-1,N); km2 = idx(k-2,N); kp1 = idx(k+1,N); kp2 = idx(k+2,N)
    sdiff = snew - grid[:,i,j,k]
    -pars.B::Float64 * sdiff[3] -
    pars.J::Float64  * dot(sdiff, grid[:,im1,j,k] + grid[:,ip1,j,k] + grid[:,i,jm1,k] + grid[:,i,jp1,k] + grid[:,i,j,km1] + grid[:,i,j,kp1]) +
    pars.Jp::Float64 * dot(sdiff, grid[:,im2,j,k] + grid[:,ip2,j,k] + grid[:,i,jm2,k] + grid[:,i,jp2,k] + grid[:,i,j,km2] + grid[:,i,j,kp2]) -
    pars.K::Float64  * (crossX(sdiff, grid[:,ip1,j,k] - grid[:,im1,j,k]) +
                        crossY(sdiff, grid[:,i,jp1,k] - grid[:,i,jm1,k]) +
                        crossZ(sdiff, grid[:,i,j,kp1] - grid[:,i,j,km1])) +
    pars.Kp::Float64 * (crossX(sdiff, grid[:,ip2,j,k] - grid[:,im2,j,k]) +
                        crossY(sdiff, grid[:,i,jp2,k] - grid[:,i,jm2,k]) +
                        crossZ(sdiff, grid[:,i,j,kp2] - grid[:,i,j,km2]))
end

## Simulated Annealing

In [ ]:
function scheduleNext(anneal::Schedule)
    (anneal.os += 1) > length(anneal.T) + length(anneal.B) - 1 && return false
    anneal.os > length(anneal.T) ? (pars.T = anneal.T[anneal.os]) : pars.B = anneal.B[anneal.os % length(anneal.T)]
    return true
end

In [ ]:
@inline acceptProb(del::Float64, T::Float64) = exp(-del/T)

In [ ]:
function acceptMove!(grid::Array{Float64,4}, snew::Vector{Float64}, dE::Float64, i::Int64, j::Int64, k::Int64)
    mon.Nacc::Int64 += 1
    mon.nacc::Int64 += 1
    res.Etmp::Float64 += dE
    res.Mtmp::Vector{Float64} += snew - grid[:,i,j,k]
    grid[:,i,j,k] = snew
end

In [ ]:
function update!(grid::Array{Float64,4}, snew::Vector{Float64})
    i,j,k = rand(1:pars.N::Int64, 3)
    uniformS2!(snew)
    dE = energyChange(grid, snew, i, j, k)
    if dE < 0.0 || acceptProb(dE, pars.T::Float64) >= rand()
        acceptMove!(grid, snew, dE, i, j, k)
    else
        mon.Nrej::Int64 += 1
    end
end

In [ ]:
@inline function sweep!(grid::Array{Float64,4}, snew::Vector{Float64})
    for i=1:pars.Ntot
        update!(grid, snew)
    end
end

In [ ]:
function nextconfig!(grid::Array{Float64,4})
    snew = zeros(Float64, 3)
    for j=1:pars.Nsweeps
        sweep!(grid, snew)
    end
end

In [ ]:
function thermalize!(grid::Array{Float64,4})
    info("Start thermalizing...")
    snew = zeros(Float64, 3)
    @showprogress 1 "Thermalizing..." for i=1:pars.Ntherm::Int64
        sweep!(grid, snew)
    end
    info("Finished thermalizing.")
    grid
end

function thermalize()
    grid = initRandomGrid()
    thermalize!(grid)
end

In [ ]:
function annealing!(grid::Array{Float64,4}, anneal::Schedule)
    info("Start annealing...")
    while true
        nextconfig!(grid)
        scheduleNext(anneal) || break
    end
    info("Finished the annealing schedule.")
end

In [ ]:
function run!(grid::Array{Float64,4}, filename::ASCIIString, anneal::Schedule)
    info("Start run...")
    thermalize!(grid)
    annealing!(grid, anneal)
    h5open(getPath(filename), "w") do file
        conf, avg = initOutput(file)
        snapshot(grid, conf, avg)
        @showprogress 1 "Compute configurations..." for i = 1:pars.Nconfig::Int64-1
            nextconfig!(grid)
            snapshot(grid, conf, avg)
        end
        finalOutput(file, avg)
    end
    info("Finished run.")
    grid
end

@inline run(filename::ASCIIString, anneal::Schedule) = run!(initRandomGrid(), filename, anneal)

## I/O

In [ ]:
function snapshot(grid::Array{Float64,4}, conf::HDF5Dataset, avg::HDF5Dataset)
    res.os::Int64 += 1
    res.E[res.os] = res.Etmp::Float64
    res.M[:,res.os] = res.Mtmp::Vector{Float64}
    res.T[res.os] = pars.T::Float64
    mon.accRate[res.os] = mon.nacc::Int64 / (pars.Ntot::Int64 * pars.Nsweeps::Int64)
    mon.nacc::Int64 = 0
    appendConfig(grid, res.os, conf)
    addConfig(grid, avg)
end

In [ ]:
function initOutput(file::HDF5File)
    conf = d_create(file, "configs", datatype(Float64), dataspace(3,pars.N,pars.N,pars.N,pars.Nconfig),"chunk",(3,pars.N,pars.N,pars.N,1))
    avg = d_create(file, "avg", datatype(Float64), dataspace(3,pars.N,pars.N,pars.N))
    info("Created output file.")
    (conf, avg)
end

In [ ]:
function finalOutput(file::HDF5File, avg::HDF5Dataset)
    file["energy"] = res.E
    file["magnetization"] = res.M
    file["temperature"] = res.T
    file["acceptancerate"] = mon.accRate
    writeParameters(file)
    averageConfigs!(avg)
    writeResultAndError(file, res.E, "E_avg")
    writeResultAndError(file, getX(res.M), "Mx_avg")
    writeResultAndError(file, getZ(res.M), "My_avg")
    writeResultAndError(file, getZ(res.M), "Mz_avg")
    writeResultAndError(file, res.E, "specific_heat", fun=specificHeat)
    writeResultAndError(file, getZ(res.M), "susceptibility", fun=susceptibility)
end

In [ ]:
function writeParameters(file::HDF5File)
    file["parameters/N"] = pars.N
    file["parameters/Ntherm"] = pars.Ntherm
    file["parameters/Nsweeps"] = pars.Nsweeps
    file["parameters/Nconfig"] = pars.Nconfig
    file["parameters/J"] = pars.J
    file["parameters/Jp"] = pars.Jp
    file["parameters/K"] = pars.K
    file["parameters/Kp"] = pars.Kp
end

In [ ]:
@inline function writeResultAndError(file::HDF5File, res::Float64, err::Float64, dset::ASCIIString)
    file[dset] = [res, err]
end

function writeResultAndError(file::HDF5File, f::Vector{Float64}, dset::ASCIIString; fun::Function=mean)
    res = fun(f)
    err = sqrt(jackknife(fun, f))
    writeResultAndError(file, res, err, dset)
end

In [ ]:
@inline function averageConfigs!(avg::HDF5Dataset)
    avg[:,:,:,:] /= pars.Nconfig
end

In [ ]:
@inline function appendConfig(grid::Array{Float64,4}, os::Int64, conf::HDF5Dataset)
    conf[:,:,:,:,os] = grid
end

In [ ]:
@inline function addConfig(grid::Array{Float64,4}, avg::HDF5Dataset)
    avg[:,:,:,:] += grid
end

In [ ]:
@inline getPath(filename::ASCIIString; ext::ASCIIString="h5") = string("../../data/", filename, ".", ext)

In [ ]:
function loadData(filename::ASCIIString, dset::ASCIIString; group=nothing)
    h5open(getPath(filename), "r") do file
        if group == nothing
            idx = dset
        else
            idx = string(group, "/", dset)
        end
        data = read(file[idx])
    end
end

In [ ]:
function loadParameters(filename::ASCIIString)
    h5open(getPath(filename), "r") do f
        pars.N = read(f["parameters/N"])
        pars.Ntot = pars.N^3
        pars.Nconfig = read(f["parameters/Nconfig"])
        pars.Nsweeps = read(f["parameters/Nsweeps"])
        pars.Ntherm = read(f["parameters/Ntherm"])
        pars.T = read(f["temperature"])[1]
    end
end

## Postprocessing

### Basic Statistics

In [ ]:
function autocorr(f::Vector{Float64})
    mean, var = mean_and_var(f)
    N = length(f)
    fm = f - mean
    auto = zeros(Float64, N)
    for t=0:N-1
        tmp = 0.0
        for i=1:N-t
            tmp += fm[i]*fm[i+t]
        end
        auto[t+1] = tmp / ((N-t)*var)
    end
    auto
end

In [ ]:
function intAutoTime(f::Vector{Float64}, N::Int64; isautocorr=false)
    isautocorr || (f = autocorr(f))
    0.5 + sum(f[2:N])
end

@inline intAutoTime(f::Vector{Float64}; isautocorr=false) = intAutoTime(f, length(f), isautocorr=isautocorr)

In [ ]:
function jackknife(obs::Function, f::Vector{Float64})
    N = length(f)
    jack = [obs([f[1:i]; f[i+2:N]]) for i=0:N-1]
    (N-1) * mean( (jack - mean(jack)).^2 )
end

@inline jackknife(f::Vector{Float64}) = jackknife(mean, f)

### Skyrmion Specifics

In [ ]:
@inline getX(grid::Array{Float64}) = squeeze(grid[1,:],1)
@inline getY(grid::Array{Float64}) = squeeze(grid[2,:],1)
@inline getZ(grid::Array{Float64}) = squeeze(grid[3,:],1)
@inline getX(grid::Array{Float64,4}) = squeeze(grid[1,:,:,:],1)
@inline getY(grid::Array{Float64,4}) = squeeze(grid[2,:,:,:],1)
@inline getZ(grid::Array{Float64,4}) = squeeze(grid[3,:,:,:],1)

In [ ]:
function drawProj(f::Array{Float64,3}, dir::Int64; name::ASCIIString="proj", saveimg=false)
    proj = squeeze(sum(f, dir), dir)
    img = grayim( (proj/maximum(proj[:])).^6 )
    saveimg && save(getPath(string(name, "_", dir), extension="png"), img)
    img
end

function drawProj(f::Array{Float64,3}; name::ASCIIString="proj", save=false)
    [drawProj(f, dir, name=name, save=save) for dir=1:3]
end

In [ ]:
function braggIntensity(grid::Array{Float64,4})
    Sk = fftVF(grid)
    I = vcat(fld(pars.N,2)+1:pars.N, 1:fld(pars.N,2))
    [norm(Sk[:,i,j,k])::Float64 for i=I, j=I, k=I]
end

In [ ]:
function fftVF(grid::Array{Float64,4})
    Sx = fft3(getX(grid), pars.N::Int64)
    Sy = fft3(getY(grid), pars.N::Int64)
    Sz = fft3(getZ(grid), pars.N::Int64)
    S = zeros(Complex{Float64}, size(grid))
    # TODO: this should be doable with smart stacking and reshape
    for i=1:pars.Ntot::Int64
        @inbounds S[:,i] = [Sx[i] Sy[i] Sz[i]]
    end
    S
end

In [ ]:
function fft3(f::Array{Float64,3}, N::Int64)
    FFTW.set_num_threads(4)
    fft(f - mean(f[:]))
end

In [ ]:
@inline specificHeat(f::Vector{Float64}) = var(f)/(pars.Ntot * pars.T^2)

In [ ]:
@inline susceptibility(f::Vector{Float64}) = var(f)/(pars.Ntot * pars.T)

## Run the Simulation

In [ ]:
anneal = Schedule([0.8], [0.1])
pars = Parameters(anneal, N=10, Ntherm=10, Nsweeps=10, Nconfig=10)
mon, res = init(pars);
pars.T, pars.B, pars.N, pars.Ntherm, pars.Nconfig

In [ ]:
@time run("N$(pars.N)_T$(pars.T)_B$(pars.B)", anneal);

## Analyse the Results

### First Plots of a Single Run

In [ ]:
using PyPlot

In [ ]:
tMC = 1:pars.Nconfig
plot(res.E)
figure()
plot(tMC, getX(res.M), tMC, getY(res.M), tMC, getZ(res.M))

In [ ]:
plot(autocor(res.E,0:pars.Nconfig-1))
figure()
plot(tMC, autocor(getX(res.M),0:pars.Nconfig-1),tMC, autocor(getY(res.M),0:pars.Nconfig-1),tMC, autocor(getZ(res.M),0:pars.Nconfig-1))

In [ ]:
plot(mon.accRate)

### Bragg Intensity

In [ ]:
using Images

In [ ]:
name = "highres/T0.8_B0.1_cmpct"
avg = loadData(name, "avg");
bragg = braggIntensity(avg);

In [ ]:
drawProj(bragg, 1, saveimg=false)

### Combine Multiple Runs

In [ ]:
# Deprecated
function plotSpecificHeat(Ts, Bs)
    d = zeros(length(Ts),length(Bs),2)
    for (i,T) in enumerate(Ts), (j,B) in enumerate(Bs)
        name = "highres/T$(T)_B$(B)_cmpct"
        loadParameters(name)
        tmp = loadData(name, "energy")
        d[i, j, :] = [specificHeat(tmp), sqrt(jackknife(specificHeat, tmp))]
    end
    fig,ax = PyPlot.subplots()
    for (i,B) in enumerate(Bs)
        ax[:errorbar](Ts[4:end], d[4:end,i,1] + (i-1)*0.5, yerr=d[4:end,i,2], fmt=".", label="B=$(B)")
    end
    ax[:legend](loc="best")
end

# Deprecated
function plotSusceptibility(Ts, Bs)
    d = zeros(length(Ts),length(Bs),2)
    for (i,T) in enumerate(Ts), (j,B) in enumerate(Bs)
        name = "highres/T$(T)_B$(B)_cmpct"
        loadParameters(name)
        tmp = getZ(loadData(name, "magnetization"))
        d[i, j, :] = [susceptibility(tmp), sqrt(jackknife(susceptibility, tmp))]
    end
    fig,ax = PyPlot.subplots()
    for (i,B) in enumerate(Bs)
        ax[:errorbar](Ts[4:end], d[4:end,i,1] + (i-1)*0.5, yerr=d[4:end,i,2], fmt=".", label="B=$(B)")
    end
    ax[:legend](loc="best")
end

function plotResult(Ts, Bs, dset)
    data = [loadData("highres/T$(T)_B$(B)_cmpct", dset) for T in Ts, B in Bs]
    fig,ax = PyPlot.subplots()
    for (i,B) in enumerate(Bs)
        val = [data[j,i][1] for j in eachindex(Ts)]
        err = [data[j,i][2] for j in eachindex(Ts)]
        ax[:errorbar](Ts[5:end], val[5:end] + (i-1) * 0.5, yerr=err[5:end], fmt="o", label="B=$(B)")
    end
    ax[:legend](loc="best")
end

In [ ]:
Ts = 0.0:0.1:3.0
Bs = [0.0 0.01 0.02 0.05 0.1 0.2 0.3 0.4]
plotResult(Ts, Bs, "specific_heat")

In [ ]:
plotResult(Ts, Bs, "susceptibility")

## Tests

In [ ]:
using Base.Test

In [ ]:
function testuniformS2()
    N = 50
    dims = (N, N, N)
    grid = uniformS2(dims);
    for i=1:*(dims...)
        @test norm(grid[:,i]) ≈ 1.0
    end
    println("finished test: uniform points on S2")
end

In [ ]:
function test()
    testuniformS2()
    println("If you do not see any errors, all tests passed.")
end